In [48]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17411 sha256=c6229ce853480a569220d551037454048173ff5f3ec8071e2abb56a0b75928ac
  Stored in directory: c:\users\user2\appdata\local\pip\cache\wheels\60\b3\27\d8aff3e2d5c2d0d97a117cdf0d5f13cd121e2c2b5fb49b55a0
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0


In [49]:
import pandas as pd
import numpy as np
from googletrans import Translator

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, manhattan_distances, euclidean_distances

In [50]:
data = pd.read_csv("data/filipino_recipe_clean.csv")

In [51]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(data['ingredients_clean'])

In [52]:
translator = Translator()
text = "bangus, milkfish, garlic, oil"
text = translator.translate('bangus').text
test = tf.transform([])

AttributeError: 'NoneType' object has no attribute 'group'

### Cosine Similarity

- cosine similarity recommendations makes more sense compared to two similarity metrics

In [27]:
test_cosine = cosine_similarity(tfidf_matrix, test)
recommend_cosine = sorted(range(len(test_cosine)), key = lambda sub: test_cosine[sub])[-5:]

In [28]:
for i in recommend_cosine:
    print(data.iloc[i])
    print("---------------------------------------")

url                    https://www.kawalingpinoy.com/relyenong-bangus/
food                                                  Relyenong Bangus
course                                                            none
ingredients          1 large whole bangus milkfish, juice of 1 lemo...
instructions         Scale fish and remove innards, leaving belly i...
ingredients_clean    bangus milkfish juice lemon soy sauce onion ga...
ingredients_token    ['bangus', 'milkfish', 'juice', 'lemon', 'soy'...
Name: 2159, dtype: object
---------------------------------------
url                  https://panlasangpinoy.com/tinolang-bangus-rec...
food                                            Tinolang Bangus Recipe
course                                                            none
ingredients          1 large bangus milkfish, scales and innards re...
instructions         Heat the oil in a cooking pot.\r\nSaute the on...
ingredients_clean    bangus milkfish innards malunggay hot chayote ...
ingredients

### Manhattan Distance

In [13]:
test_manhattan = manhattan_distances(tfidf_matrix, test)
recommend_manhattan = sorted(range(len(test_manhattan)), key = lambda sub: test_manhattan[sub])[:5]

In [14]:
for i in recommend_manhattan:
    print(data.iloc[i])
    print("---------------------------------------")

url                  https://panlasangpinoy.com/filipino-food-fried...
food                                              Fried Tilapia Recipe
course                                                            none
ingredients          2 pieces tilapia cleaned and scales removed, 2...
instructions         Rub salt all over the fish including the insid...
ingredients_clean                                              tilapia
ingredients_token                                          ['tilapia']
Name: 315, dtype: object
---------------------------------------
url                      https://www.kawalingpinoy.com/tortang-talong/
food                                                    Tortang Talong
course                                                       Side Dish
ingredients          4 eggplants, 4 eggs, salt and pepper to taste....
instructions         With a knife or fork, prick eggplant. Arrange ...
ingredients_clean                                         eggplant egg
ingredients_

### Euclidean Distance

In [11]:
test_euclid = euclidean_distances(tfidf_matrix, test)
recommend_euclid = sorted(range(len(test_euclid)), key = lambda sub: test_euclid[sub])[:5]

In [12]:
for i in recommend_euclid:
    print(data.iloc[i])
    print("---------------------------------------")

url                  https://panlasangpinoy.com/tochong-bangus-recipe/
food                                                    Tochong Bangus
course                                                     Main Course
ingredients          1 bangus cleaned and sliced diagonally into pi...
instructions         Rub salt all over the fish. Let it stay for 10...
ingredients_clean    bangus tofu tahure ferment bean curd mash taus...
ingredients_token    ['bangus', 'tofu', 'tahure', 'ferment', 'bean'...
Name: 71, dtype: object
---------------------------------------
url                      https://www.kawalingpinoy.com/totsong-bangus/
food                                                    Totsong Bangus
course                                                            none
ingredients          1 whole bangus, scaled, gutted and cut into se...
instructions         Rinse fish and drain well. Season with salt an...
ingredients_clean    bangus canola tofu onion garlic ginger rom tom...
ingredients_t